In [6]:
import firebase_admin
from firebase_admin import credentials, firestore
import weaviate
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display
from llama_index.vector_stores.weaviate import WeaviateVectorStore

In [2]:
cred = credentials.Certificate("rate-my-professor-50aba-firebase-adminsdk-zvwp8-22c0991e41.json")  # Replace with your Firestore service account key
firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
client = weaviate.Client(
    url="https://69w1cdbysvs5vblwcjfiqq.c0.us-east1.gcp.weaviate.cloud", 
    auth_client_secret=weaviate.AuthApiKey(api_key = "bCbSXLReNDNqf3WS6CeVevDnvZDAUkBB6H7s",)
    )

c:\Python312\Lib\site-packages\weaviate\__init__.py:143: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
c:\Python312\Lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [4]:
def get_professors_data():
    professors_ref = db.collection('professors')
    docs = professors_ref.stream()
    data = []
    for doc in docs:
        data.append(doc.to_dict())  # Each document is added to the list
    return data

In [5]:
data = get_professors_data()

In [7]:
from llama_index.core import StorageContext

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="Review"
)

In [12]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [10]:
print(vector_store)
print(storage_context)

stores_text=True is_embedding_query=True index_name='Review' url=None text_key='text' auth_config={} client_kwargs={}
StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x000001E3C3603020>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x000001E3C36036B0>, vector_stores={'default': WeaviateVectorStore(stores_text=True, is_embedding_query=True, index_name='Review', url=None, text_key='text', auth_config={}, client_kwargs={}), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x000001E3C3602E70>, property_graph_store=None)


In [20]:
from llama_index.core import Document as LlamaDocument

class Document(LlamaDocument):
    def __init__(self, text, metadata):
        super().__init__(text=text, metadata=metadata)
        # Convert the hash to a string for the id_
        self.id_ = str(hash(text)) 

In [26]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# Configure the embedding model directly on the Settings object
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [30]:
def create_index_from_dict(data):
    documents = [
        Document(
            text=item['review'],
            metadata={
                'name': item['name'],
                'specialization': item['specialization'],
                'review': item['review']
            }
        )
        for item in data
    ]
    
    # Debugging: Print the documents to verify their structure
    for doc in documents:
        print(f'Document ID: {doc.id_}, Text: {doc.text}, Metadata: {doc.metadata}')
    
    return VectorStoreIndex.from_documents(documents)

In [31]:
index = create_index_from_dict(data)

Document ID: 8756015178825492436, Text: Teaches operating systems with great depth. Lectures are very engaging and interactive., Metadata: {'name': 'Dr. Michael Johnson', 'specialization': 'Operating Systems', 'review': 'Teaches operating systems with great depth. Lectures are very engaging and interactive.'}
Document ID: -2742461801327301193, Text: Excellent professor for computer networks. Classes are difficult but very insightful., Metadata: {'name': 'Dr. Sarah Wilson', 'specialization': 'Computer Networks', 'review': 'Excellent professor for computer networks. Classes are difficult but very insightful.'}
Document ID: -1692536283392036, Text: Her data structures course is well-organized. She’s always willing to help students outside class., Metadata: {'name': 'Dr. Karen Thompson', 'specialization': 'Data Structures', 'review': 'Her data structures course is well-organized. She’s always willing to help students outside class.'}
Document ID: -785108643325655933, Text: Great at explain

In [36]:
from llama_index.llms.openrouter import OpenRouter

# Initialize OpenRouter with your API key
llm = OpenRouter(api_key="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", max_tokens=256, context_window=4096, model="meta-llama/llama-3-8b-instruct:free")

In [37]:

query_engine = index.as_query_engine(llm=llm)

# Now you can query the engine
response = query_engine.query("Tell me about a professor who specializes in AI.")
print(response)

Dr. James Lee is a professor who is very passionate about artificial intelligence. His projects are designed to prepare students for the industry.


In [38]:
display(Markdown(f"{response}"))

Dr. James Lee is a professor who is very passionate about artificial intelligence. His projects are designed to prepare students for the industry.